In [1]:
import pandas as pd

In [9]:
train=pd.read_csv('data/fraudTrain.csv',index_col=None)
test=pd.read_csv('data/fraudTest.csv',index_col=None)

In [10]:
## we will use 23 months for training and 1 month for testing 
## we will have some static feats
## we will have some dynamic feats for each time shots
##

In [11]:
total_data=pd.concat((train,test))
total_data.head(1)

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0


In [12]:
total_data.trans_date_trans_time.max(),total_data.trans_date_trans_time.min()

('2020-12-31 23:59:34', '2019-01-01 00:00:18')

In [17]:
def get_discrete_time(time):
    year=int(time[:4])
    month=int(time[5:7])
    if year>2019:
        month+=12
    return month

In [18]:
total_data['discrete_time']=total_data['trans_date_trans_time'].apply(lambda x: get_discrete_time(x) )

In [19]:
total_data.discrete_time.max(),total_data.discrete_time.min()

(24, 1)